In [1]:
#!/usr/bin/env python3
import tensorflow as tf

import os

from src.ForecastModel.utils.metrics import calculate_nse, calculate_kge, calculate_bias, calculate_rms
from src.ForecastModel.utils.losses import loss_peak_mse
from src.ForecastModel.utils.postprocessing import ModelHandler, df2latex, get_bold_mask, load_metrics

import numpy as np
import pandas as pd

In [2]:
PLOT_PATH          = r"plots"
DATA_PATH          = r"src\data\Dataset.csv"
CROSS_INDICES_PATH = r"src\data\indices"

In [3]:
models = {
    "arima": ModelHandler("ARIMA",
                r"rst\ARIMA",
                is_final_model = True,
                is_external_model = True,
                color = "#E69F00",
                ls = "--",
                  ),
    "arimax": ModelHandler("ARIMAX",
                r"rst\ARIMAX",
                is_final_model = True,
                is_external_model = True,
                color = "#0072B2",
                ls = "--",
                  ),
     "pbhm-hlstm": ModelHandler("PBHM-HLSTM",
                   r"rst\PBHM-HLSTM",
                   is_final_model = True,
                   color = "#56B4E9",
                   ls = "-",
                 ),
     "elstm": ModelHandler("eLSTM",
                   r"rst\eLSTM",
                   is_final_model = True,
                   color = "#D55E00",
                   ls = "-",
                 ),
     "lstm": ModelHandler("LSTM",
                   r"rst\LSTM",
                   is_final_model = True,
                   color = "#CC79A7",
                   ls = "-",
                 ),
     }


# Table 1: Statistics of the catchment

In [4]:
# calculate metrics of hydrologic model per fold

dfh = pd.read_csv(DATA_PATH, parse_dates=["time"])

hyd_metrics = {}
for year in range(2011, 2018):
      dfp = dfh.loc[dfh["time"].dt.year == year]
      hyd_metrics[year] = {
          "kge":  calculate_kge(dfp.qmeasval.values, dfp.qsim.values),
          "nse":  calculate_nse(dfp.qmeasval.values, dfp.qsim.values),
          "bias": calculate_bias(dfp.qmeasval.values, dfp.qsim.values),
          "q_mean": dfp.qmeasval.mean(),
          "q_std":  dfp.qmeasval.std(),
          "q_max":  dfp.qmeasval.max(),
          "q_sum":  dfp.qmeasval.sum()*0.25*60*60 / 1e6,
          "qs_mean": dfp.qsim.mean(),
          "qs_std":  dfp.qsim.std(),
          "qs_max":  dfp.qsim.max(),
          "qs_sum":  dfp.qsim.sum()*0.25*60*60 / 1e6,
          "pmx_max": dfp.pmax.max()*4,
          "pmx_sum": dfp.pmax.sum(),
          "p_max":   dfp.pmean.max()*4,
          "p_sum":   dfp.pmean.sum(),
          "t_mean": dfp.tmean.mean(),
          "t_std":  dfp.tmean.std(),
      }    


In [5]:
df_catchment_stats = pd.DataFrame(hyd_metrics).transpose()
df_catchment_stats = df_catchment_stats[df_catchment_stats.columns[3:]].transpose()
df_catchment_stats.columns = [str(x) for x in df_catchment_stats.columns]
df_catchment_stats

,2011,2012,2013,2014,2015,2016,2017
q_mean,0.571149,1.009310,1.208354,1.167666,0.711204,0.829312,0.567430
q_std,0.245171,0.842681,0.681078,0.669319,0.332219,0.682278,0.229892
q_max,9.610000,25.200000,15.000000,7.270000,5.850000,17.940000,9.210000
q_sum,18.011757,31.916797,38.106657,36.823518,22.428531,26.224821,17.698878
qs_mean,0.617823,1.103376,1.399735,1.349319,0.755306,0.916051,0.907808
qs_std,0.331923,0.832882,1.016826,0.719722,0.519150,0.701360,0.479473
qs_max,4.195333,8.937861,11.354018,7.681351,4.502031,6.426838,4.611206
qs_sum,19.483660,34.891398,44.142029,42.552124,23.819337,28.967737,28.315708
pmx_max,118.720001,180.520004,100.480003,84.639999,109.919998,231.880005,173.360001
pmx_sum,2159.559998,2877.670002,2777.400000,2847.860000,2198.090000,3222.800001,3076.049996


In [6]:
df2latex(df_catchment_stats, os.path.join(PLOT_PATH, r"table_1_summary_data.txt"))

plots\table_1_summary_data.txt


# Table 2: Average Model Performance

In [7]:
metric_names = ["kge", "nse", "bias"]
metric_labels = ["KGE", "NSE", "PBIAS"]

xx = np.arange(1,97)
df = pd.DataFrame(columns=["name", "year"])

n_row = -5

for n, key in enumerate(models.keys()):
    metrics          = load_metrics(os.path.join(models[key].lg_path, "metrics.txt"))
    metrics_baseline = load_metrics(os.path.join(models["arima"].lg_path, "metrics.txt"))
    
    n_row += 5
    
    for j, met in enumerate(metric_names):

        metric_test     = metrics["test"][met]
        metric_baseline = metrics_baseline["test"][met]
        
        for i in range(5):
            df.loc[n_row+i, ["name", "year"]] = [models[key].name, 2013+i]

            # evaluations --------------------------------------------------------------------------
            # median metrics of test set
            df.loc[n_row+i, [f"{met}_test"]]   = [np.median(metric_test[i])]
            # metric for inital and final step in forecasting window
            df.loc[n_row+i, [f"{met}_init"]]   = [metric_test[i][0]]
            df.loc[n_row+i, [f"{met}_final"]]  = [ metric_test[i][-1]]
            # difference of metric between inital and final step in forecasting window
            df.loc[n_row+i, [f"{met}_drop"]]   = [metric_test[i][0] - metric_test[i][-1]]
            # num of lead time steps required so that model outperforms the baseline (ARIMA)
            if met != "bias":
                out_perform_idxs = [n for n, (x,y) in enumerate(zip(metric_baseline[i], metric_test[i])) if y-x > 0]
                if len(out_perform_idxs) > 0:
                    df.loc[n_row+i, [f"{met}_out"]] = out_perform_idxs[0]
                    df.loc[n_row+i, [f"{met}_out_hours"]] = out_perform_idxs[0] * 0.25
                else:
                    df.loc[n_row+i, [f"{met}_out"]] = -1
                    df.loc[n_row+i, [f"{met}_out_hours"]] = -1

# summary
cols = df.columns.to_list()
df[cols[:2]+cols[4:9]+cols[12:17]+cols[20:]]

,name,year,kge_final,kge_drop,kge_out,kge_out_hours,nse_test,nse_out,nse_out_hours,bias_test,bias_init,bias_final
0,ARIMA,2013,0.800486,0.193120,-1.0,-1.00,0.648435,-1.0,-1.00,-0.027993,-0.000717,0.053995
1,ARIMA,2014,0.811550,0.183926,-1.0,-1.00,0.701398,-1.0,-1.00,0.033416,0.000910,0.043052
2,ARIMA,2015,0.745181,0.250766,-1.0,-1.00,0.662314,-1.0,-1.00,0.075640,0.001628,0.104698
3,ARIMA,2016,0.815479,0.178589,-1.0,-1.00,0.690027,-1.0,-1.00,-0.018591,-0.000369,-0.028030
4,ARIMA,2017,0.517551,0.462838,-1.0,-1.00,-0.076100,-1.0,-1.00,-0.355783,-0.007793,-0.511570
5,ARIMAX,2013,0.784874,0.208520,5.0,1.25,0.646632,4.0,1.00,-0.013866,-0.000224,-0.023260
6,ARIMAX,2014,0.785025,0.210125,-1.0,-1.00,0.660631,-1.0,-1.00,0.025766,0.000428,0.039924
7,ARIMAX,2015,0.735858,0.259767,1.0,0.25,0.662379,2.0,0.50,0.036123,0.000689,0.061743
8,ARIMAX,2016,0.795679,0.197371,4.0,1.00,0.652562,1.0,0.25,-0.010952,-0.000143,-0.019201
9,ARIMAX,2017,0.449949,0.528284,-1.0,-1.00,-0.235030,0.0,0.00,-0.191906,-0.003196,-0.326225


In [8]:
# calculate metrics of hydrologic model per fold
dfh = pd.read_csv(DATA_PATH, parse_dates=["time"])

hyd_metrics = {}
for year in range(2012, 2018):
      dfp = dfh.loc[dfh["time"].dt.year == year]
      hyd_metrics[year] = {
          "kge":  calculate_kge(dfp.qmeasval.values, dfp.qsim.values),
          "nse":  calculate_nse(dfp.qmeasval.values, dfp.qsim.values),
          "bias": calculate_bias(dfp.qmeasval.values, dfp.qsim.values),
      }    
    
    
# averaged metrics
evalu = {}
for n, key in enumerate(models.keys()):
    metrics = load_metrics(os.path.join(models[key].lg_path, "metrics.txt"))
    evalu[key] = {}
    for i in range(0,5):
        year = 2013 + i
        evalu[key][year] = {}
        for j, met in enumerate(["kge", "nse", "bias"]):
            metric = metrics["test"][met][i]
            evalu[key][year][met] = metric
       

In [9]:
years = np.array([x for x in range(2013,2018)])
metric_names = ["kge", "nse", "bias"]

df_avg_model = pd.DataFrame(index=years)

# get metrics from the hydrological model
for year in years:
    for met in metric_names:
        df_avg_model.loc[year, f"hyd_{met}"] = hyd_metrics[year][met]

# get metrics from data models
for n,key in enumerate(evalu.keys()):
    for year in years:
        for met in metric_names:
            df_avg_model.loc[year, f"{key}_{met}"] = np.mean(evalu[key][year][met])

df_avg_model

,hyd_kge,hyd_nse,hyd_bias,arima_kge,arima_nse,arima_bias,arimax_kge,arimax_nse,arimax_bias,pbhm-hlstm_kge,pbhm-hlstm_nse,pbhm-hlstm_bias,elstm_kge,elstm_nse,elstm_bias,lstm_kge,lstm_nse,lstm_bias
2013,0.631525,0.185013,15.838105,0.841647,0.686484,-0.020626,0.839583,0.681185,-0.013552,0.869530,0.906583,4.508858,0.894369,0.845648,1.084802,0.570578,0.080617,12.379312
2014,0.738640,0.494420,15.556921,0.851671,0.739703,0.031338,0.832161,0.706090,0.024193,0.916186,0.951527,-4.115020,0.853864,0.902426,-2.061313,0.564214,0.643576,3.048446
2015,0.505049,0.236257,6.201059,0.820567,0.701578,0.069297,0.827602,0.694920,0.034544,0.861502,0.897470,-7.809336,0.877162,0.901531,-1.307625,0.307356,-0.095721,31.559686
2016,0.738958,0.512468,10.459236,0.854126,0.714077,-0.017186,0.843543,0.683436,-0.010505,0.850701,0.857629,7.419091,0.679236,0.684377,8.587589,0.520909,0.522718,-2.335129
2017,0.191555,-4.243535,59.985891,0.564865,0.022320,-0.326514,0.515749,-0.114457,-0.183182,0.821199,0.609330,-13.001763,0.786206,0.638627,0.722015,0.483421,-1.135756,37.034761


In [10]:
mask = get_bold_mask(df_avg_model.abs(), [np.argmax, np.argmax, np.argmin], 3, 0)
df2latex(df_avg_model[df_avg_model.columns[[0,1,2,3,4,5,6,7,8,9,10,11]]], os.path.join(PLOT_PATH, r"table_2_mean_metrics.txt"), mask, '6.2f')

plots\table_2_mean_metrics.txt


# Table 4: Peak Performance

In [11]:
def dt(dates, format="%d/%m/%Y %H:%M"):
    if dates.tz == None:
        # make TZ aware
        return pd.to_datetime(dates, format=format).tz_localize("Europe/London").tz_convert("UTC")
    else:
        return pd.to_datetime(dates, format=format).tz_convert("UTC")

In [12]:
# def
num_peaks_per_fold = 2    # number of peaks per fold to analyze
load_predictions   = True # load predictions or newly predict with models

In [13]:
idx = -10
dfp = pd.DataFrame(columns = ["name", "year", "peak", 
                              "peak_flow", "total_flow",
                              "hyd_perr", "hyd_poff",
                              "rms_hyd", "flow_hyd", 
                              "rms_0", "rms_m", "rms_95",
                              "flow_0", "flow_m", "flow_95"]
                   
                        ) 
dfp = dfp.astype(dtype= {"name"     :"str",     "year"      :"int32",    "peak"  : "int32", 
                         "peak_flow":"float64", "total_flow":"float64",
                         "hyd_perr" :"float64", "hyd_poff"  :"float64",
                         "rms_hyd"  :"float64", "flow_hyd"  :"float64", 
                         "rms_0"    :"float64", "rms_m"     :"float64", "rms_95" : "float64",
                         "flow_0"   :"float64", "flow_m"    :"float64", "flow_95": "float64"}
                )

for n, key in enumerate(models.keys()):
    idx += 10
    print(key)
    eval_path = os.path.join(models[key].hp_path, "eval_peaks.pkl")
    
    if not os.path.exists(eval_path) or load_predictions == False:
        eval_peaks = []

        # load datamodel
        dm = DataModelCV(DATA_PATH,
               target_name       = models[key].target_name,
               hincast_features  = models[key].feat_hindcast,
               forecast_features = models[key].feat_forecast,
             )

        if models[key].is_external_model:
            overlap_length = 0
            hindcast_length = 96
        else:
            # load trial data
            with open(os.path.join(models[key].hp_path, "trial.json")) as f:
                trial = json.load(f)

            hindcast_length = trial['hyperparameters']['values']['hindcast_length']
            try:
                overlap_length = trial['hyperparameters']['values']['osc_length']
            except:
                overlap_length = 0 

        dm.main(os.path.join(CROSS_INDICES_PATH, f"cross_indices_{hindcast_length}.pkl"))

        for n_fold in dm.cross_sets.keys():
            year = 2013 + n_fold

            # load dataset
            X, y  = dm.getDataSet(dm.cross_sets[n_fold]["test"], scale=True) 

            # get hydrologial model 
            s = dm.getFeatureSet(n_fold+2, "qsim")[2]
            df = pd.DataFrame({'index':dt(s.index), 'qhyd':s.values}).set_index("index")

            # add ground truth
            s = dm.getFeatureSet(n_fold+2, "qmeasval")[2]
            s.index = dt(s.index)
            df = df.merge(s.rename("qmeas").to_frame(), left_index=True, right_index=True)

            if models[key].is_external_model:
                
                ext_df = pd.read_pickle(os.path.join(models[key].hp_path, f"forecast_{year}.pkl"))

                ext_df.index = pd.date_range(ext_df.index[0], ext_df.index[-1], freq="15min", tz="UTC")
                
                forecasts_df = ext_df[[f"fc{x:d}" for x in range(96)]].copy()
                forecasts_df.columns = [f"q{x:d}" for x in range(96)]

                del ext_df
                
            else:
                # load model
                tf.keras.backend.clear_session()
                model  = tf.keras.models.load_model(os.path.join(models[key].hp_path, f"model_fold_{n_fold:d}.keras"),
                                               custom_objects={'peak_loss'    : loss_peak_mse, # dummy as no costum functions are saved by keras
                                                              'kge_nse_loss'  : loss_peak_mse, #
                                                              'loss_nkge_nnse': loss_peak_mse, #
                                                              })

                yp = model.predict(X, batch_size=1000)
                
                if key == "lstm_residual":
                    _, _, yidx = dm.sets[dm.cross_sets[n_fold]["test"]]
                    simu = dm.getWithIndexArray(["qsim"], yidx)
            
                    # get real values from residuals
                    yp += simu[:,:,0]
            
                forecasts_df = pd.DataFrame(data    = yp, 
                                        columns = [f"q{x:d}" for x in range(yp.shape[1])],
                                        index   = dt(dm.getTimeSet(n_fold+2, 0)[2]))

                # save dataframe 
                df_out = forecasts_df.copy()
                df_out.index = pd.to_datetime(df_out.index, format="%d/%m/%Y %H:%M", utc=True)
                df_out.to_pickle(os.path.join(models[key].hp_path, f"forecast_{year}.pkl"))
            
            
            # get forcasting stats                          
            for forecast_step in range(1, forecasts_df.shape[1]):
                forecasts_df[f"q{forecast_step:d}"] = forecasts_df[f"q{forecast_step:d}"].shift(forecast_step)
            
            # merge model predctions
            df = df.merge(forecasts_df, left_index=True, right_index=True)  
            
            # merge prcipitation
            s = pd.Series(dm.getFeatureSet(n_fold+2, "pmean", 0)[2].values, dt(dm.getTimeSet(n_fold+2, 0)[2]))
            df = df.merge(s.rename("pmean").to_frame(), left_index=True, right_index=True)
            
            forecasts_df.dropna(inplace=True)
            stats_df = pd.DataFrame(columns = ["fmin", "fmax", "fmean", 
                                               "fq95", "fq90", "fq75",
                                               "fq50",
                                               "fq25", "fq10", "fq5"],
                                   index = forecasts_df.index)
            
            for i, row in forecasts_df.iterrows():
                stats_df.loc[i] = [row.values.min(), row.values.max(), row.values.mean()] + \
                                        [np.quantile(row.values, float(x[2:])/100) for x in stats_df.columns[3:]]
                  
            # merge stats
            df = df.merge(stats_df, left_index=True, right_index=True)        
            df.dropna(inplace=True)
                                           
            peaks = get_n_peaks(df, "qmeas", num_peaks_per_fold, 24*4)
            peaks["n_fold"] = n_fold + 2

            # add to summary
            eval_peaks.append(peaks)

        # save data to pickle
        df = pd.concat(eval_peaks, axis=0)
        df.to_pickle(eval_path)

    df = pd.read_pickle(eval_path)
    for n_fold in df.n_fold.unique().tolist():
        #print(f"processing fold {n_fold}")
        peaks = df[df.n_fold == n_fold]
        for p in range(num_peaks_per_fold):

            # eval peaks
            idx_peak  = peaks[peaks.n_peak == p]["qmeas"].argmax()
            peak_flow = peaks[peaks.n_peak == p]["qmeas"].max()
            dfp.loc[idx+n_fold+5*p, ["hyd_perr", "hyd_poff"]] = [peaks[peaks.n_peak == p]["qhyd"].max() - peak_flow,
                                                                 peaks[peaks.n_peak == p]["qhyd"].argmax() - idx_peak,
                                                                ]
            dfp.loc[idx+n_fold+5*p, [f"perr_{x}" for x in range(96)]] = [peaks[peaks.n_peak == p][f"q{x}"].max() - peak_flow for x in range(96)]
            dfp.loc[idx+n_fold+5*p, [f"poff_{x}" for x in range(96)]] = [peaks[peaks.n_peak == p][f"q{x}"].argmax() - idx_peak for x in range(96)]

            # eval section
            rms_q0 = calculate_rms(peaks[peaks.n_peak == p]["qmeas"].values, 
                                    peaks[peaks.n_peak == p]["q0"].values)
            rms_qm = calculate_rms(peaks[peaks.n_peak == p]["qmeas"].values, 
                                    peaks[peaks.n_peak == p]["fmean"].values)
            rms_q95 = calculate_rms(peaks[peaks.n_peak == p]["qmeas"].values, 
                                    peaks[peaks.n_peak == p]["q95"].values)
            rms_hyd = calculate_rms(peaks[peaks.n_peak == p]["qmeas"].values, 
                                    peaks[peaks.n_peak == p]["qhyd"].values)
            
            peak_flow = peaks[peaks.n_peak == p]["qmeas"].max()
            
            dfp.loc[idx+n_fold+5*p, ["name", "year", "peak"]] = [models[key].name, np.int32(2011+n_fold), np.int32(p)]
            dfp.loc[idx+n_fold+5*p, ["peak_flow", "total_flow"]] = [peak_flow, peaks[peaks.n_peak == p]["qmeas"].sum()]
            dfp.loc[idx+n_fold+5*p, ["rms_hyd", "flow_hyd"]] = [rms_hyd,  peaks[peaks.n_peak == p]["qhyd"].sum()]
            dfp.loc[idx+n_fold+5*p, ["rms_0", "rms_m", "rms_95"]] = [rms_q0, rms_qm, rms_q95]
            dfp.loc[idx+n_fold+5*p, ["flow_0", "flow_m", "flow_95"]] = [peaks[peaks.n_peak == p]["q0"].sum(),
                                                                     peaks[peaks.n_peak == p]["fmean"].sum(),
                                                                     peaks[peaks.n_peak == p]["q95"].sum(),
                                                                    ]

arima
arimax
pbhm-hlstm
elstm
lstm


In [14]:
dfout = dfp[["name", "year", "peak", "peak_flow", "hyd_perr", "hyd_poff"]].copy()
dfout["peak_median"] = dfp.filter(regex='^perr').mean(axis=1)
dfout["off_median"] = dfp.filter(regex='^poff').median(axis=1)

dfout["hyd_perr"] = 100 * dfout["hyd_perr"] /  dfout["peak_flow"]
dfout["peak_median"] = 100 * dfout["peak_median"] /  dfout["peak_flow"]

dfout = dfout.reset_index()
df = dfout.loc[0:9, dfout.columns[2:7]]

for n, key in enumerate(models.keys()):
    df = df.join(dfout.loc[n*10:n*10+10, dfout.columns[7:9]].reset_index(drop=True), how='left', rsuffix="_" + models[key].name)

df = df.set_index("year", drop=True)
df

,peak,peak_flow,hyd_perr,hyd_poff,peak_median,off_median,peak_median_ARIMAX,off_median_ARIMAX,peak_median_PBHM-HLSTM,off_median_PBHM-HLSTM,peak_median_eLSTM,off_median_eLSTM,peak_median_LSTM,off_median_LSTM
year,,,,,,,,,,,,,,
2013.0,0.0,15.00,-90.340227,31.0,-61.046688,47.5,-50.924530,47.5,-30.770994,5.0,-77.233465,59.5,-92.029059,15.0
2013.0,1.0,10.02,13.313555,20.0,1.479891,19.0,-1.855905,18.0,-24.321026,-1.0,-37.633298,-1.0,-40.702521,1.0
2014.0,0.0,7.27,3.475723,18.0,5.004988,18.0,4.891835,19.0,-19.391247,-2.0,-27.581298,0.0,-42.838470,12.0
2014.0,1.0,6.23,23.296167,16.0,20.668729,16.0,21.083239,15.5,-21.440319,4.0,-28.018915,2.0,-33.101205,9.0
2015.0,0.0,5.85,-62.460701,36.0,-44.912609,36.0,-39.028931,47.5,-3.607595,1.0,-62.350330,10.0,-65.550217,13.0
2015.0,1.0,3.33,4.708865,49.0,7.519117,48.0,10.471390,49.0,-17.137426,3.0,-31.876804,0.0,-20.444024,31.0
2016.0,0.0,17.94,-73.571109,26.0,-37.055424,47.5,-23.099892,47.5,-66.765543,3.0,-75.513721,5.0,-83.632628,11.0
2016.0,1.0,9.99,-45.408064,18.0,-41.315906,18.0,-37.258700,47.5,-53.349106,70.5,-63.588233,27.5,-72.955914,11.0
2017.0,0.0,9.21,-49.932621,25.0,-35.912776,37.5,-29.220201,48.5,-7.791672,-1.0,-55.421495,3.0,-54.332874,1.0


In [15]:
# make latex table and bold best values per row and metric
mask = get_bold_mask(df.abs(), np.argmin, 2, 2)
df2latex(df, 
         os.path.join(PLOT_PATH, r"table_4_peak_compare_percent.txt"), 
         mask, 
         ['d', '5.2f', '+5.1f', 'd'] + ['+5.1f', 'd']*len(models.keys()),
        )

plots\table_4_peak_compare_percent.txt


In [16]:
df.abs().median()

peak                       0.500000
peak_flow                  8.290000
hyd_perr                  47.670343
hyd_poff                  25.500000
peak_median               36.484100
off_median                32.750000
peak_median_ARIMAX        24.546750
off_median_ARIMAX         47.500000
peak_median_PBHM-HLSTM    20.415783
off_median_PBHM-HLSTM      2.500000
peak_median_eLSTM         57.775148
off_median_eLSTM           4.000000
peak_median_LSTM          58.421538
off_median_LSTM           11.000000
dtype: float64